In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset["Purchased"]

In [9]:
indep.shape

(400, 3)

In [10]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [11]:
# split into training set and test
# in regression we skip this step in gridsearch method, but in classification we do this in order to get the confusion matrix and classification report.

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [12]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
param_grid = {'criterion':['gini', 'entropy', 'log_loss'],
              'splitter':['best', 'random'],
              'max_features':['sqrt', 'log2', None]}

grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')

# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2', None],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [14]:
# print best parameter after tuning
# print(grid.best_params_)
clf=grid.cv_results_
grid_predictions=grid.predict(X_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, grid_predictions)

# print classification report
from sklearn.metrics import classification_report
clf_report=classification_report(y_test, grid_predictions)

In [15]:
# f1 score is one of the evaluation metrics
# sample to check f1 macro value
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'gini', 'max_features': None, 'splitter': 'random'}: 0.9027686670643886


In [16]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[79  6]
 [ 7 42]]


In [17]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.92      0.93      0.92        85
           1       0.88      0.86      0.87        49

    accuracy                           0.90       134
   macro avg       0.90      0.89      0.89       134
weighted avg       0.90      0.90      0.90       134



In [18]:
# roc_auc score is also an evaluation metric (a scoring parametr in classification)
# if the roc_auc value is nearly 1, then the model performs well

from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.8932773109243698

In [19]:
table=pd.DataFrame.from_dict(clf)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002779,0.000740,0.002704,0.000933,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.829615,0.889022,0.795256,0.887907,0.868632,0.854087,0.036430,8
1,0.001105,0.000921,0.002675,0.001613,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.849794,0.866968,0.759721,0.869709,0.744535,0.818145,0.054545,18
2,0.001672,0.000648,0.001985,0.000321,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.847141,0.852030,0.833323,0.832483,0.924528,0.857901,0.034175,6
3,0.000816,0.000755,0.002677,0.000531,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.864871,0.825959,0.796687,0.770530,0.867097,0.825029,0.037766,17
4,0.001307,0.001083,0.002722,0.001074,gini,None,best,"{'criterion': 'gini', 'max_features': None, 's...",0.829615,0.868752,0.833323,0.870362,0.943041,0.869019,0.040763,4
5,0.000812,0.000994,0.002830,0.001584,gini,None,random,"{'criterion': 'gini', 'max_features': None, 's...",0.867478,0.889022,0.796284,0.962636,0.903610,0.883806,0.053964,1
6,0.002016,0.000015,0.002239,0.000460,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.804764,0.828959,0.832483,0.851527,0.905069,0.844560,0.033713,10
7,0.002022,0.000620,0.002678,0.000937,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.864871,0.790949,0.777291,0.870362,0.865054,0.833706,0.040764,15
8,0.001622,0.001035,0.002506,0.000889,entropy,log2,best,"{'criterion': 'entropy', 'max_features': 'log2...",0.804764,0.811321,0.833323,0.868632,0.885265,0.840661,0.031544,12
9,0.001308,0.001256,0.002163,0.000159,entropy,log2,random,"{'criterion': 'entropy', 'max_features': 'log2...",0.808927,0.868752,0.758030,0.889033,0.867097,0.838368,0.048242,13


In [20]:
age_input=float(input("Age:"))
salary_input=float(input("Salary:"))
sex_male_input=int(input("Sex Male 0 or 1:"))

Age: 55
Salary: 35000
Sex Male 0 or 1: 1


In [21]:
Future_Prediction = grid.predict([[age_input,salary_input,sex_male_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
